**Multi Agent Customer Support Agent**
Here, Customer support agent will take user query, collect answers through some website information, and will serve user query, the answer will be sent to quality assurance agent and this agent, depending upon the quality will either approve and send back to previous sustomer support agent...We will also introduce tool which agent can use to solve user queries.... In thsi lesson we will learn about crew ai suggested prompting technique like role, goal, backstory, co-ordination and information exchange between agents, concepts of memory and guardrails to be followed

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.39.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os

# Directly set your API key and model
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"  # Use a valid model ID

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at John Deere(https://www.deere.co.in/) and "
        " are now working on providing "
		"support to {customer}, a prospective customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

By not setting allow_delegation=False, allow_delegation takes its default value of being True.
This means the agent can delegate its work to another agent which is better suited to do a particular task.

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at John Deere (https://www.deere.co.in/) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

Role Playing: Both agents have been given a role, goal and backstory.
Focus: Both agents have been prompted to get into the character of the roles they are playing.
Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

**Tools, Guardrails and Memory**

**Tools**

Import CrewAI tools

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
#  for documentation https://docs.crewai.com/en/introduction
#  for scapewebsite https://docs.crewai.com/en/tools/web-scraping/scrapewebsitetool
# This scrape website and employs RAG over it
# For creating custome tools https://docs.crewai.com/en/learn/create-custom-tools
# Source Code Github Link https://github.com/crewAIInc/crewAI-tools

*Possible Custom Tools*

Load customer data

Tap into previous conversations

Load data from a CRM

Checking existing bug reports

Checking existing feature requests

Checking ongoing tickets

... and more

Some ways of using CrewAI tools.

search_tool = SerperDevTool()

scrape_tool = ScrapeWebsiteTool()

In case You want to scrape only one Url, you can provide Url Link as argument otherwise entire website will be scraped

In [8]:
docs_scrape_tool = ScrapeWebsiteTool(url="https://www.deere.co.in/en/tractors/")

Different Ways to Give Agents Tools

Agent Level: The Agent can use the Tool(s) on any Task it performs.

Task Level: The Agent will only use the Tool(s) when performing that specific Task.

Note: Task Tools override the Agent Tools.


Creating Tasks

You are passing the Tool on the Task Level.

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with  important query:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

quality_assurance_review is not using any Tool(s)

Here the QA Agent will only review the work of the Support Agent

In [10]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is  accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, short and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


Creating the Crew

Memory

Setting memory=True when putting the crew together enables Memory.

In [11]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=1,
  memory=True
)

Running the Crew

Note: LLMs can provide different outputs for they same input, so what you

get might be different than what you see here.

Guardrails

By running the execution below, you can see that the agents and the
responses are within the scope of what we expect from them.

In [ ]:
#  inquiry 1
inputs = {
    "customer": "A progressive farmer from Hoshiarpur (≈12 acres, partly hilly)",
    "person": "Bakhtawar Singh",
    "inquiry": (
        "Sat Sri Akal. I need a tractor with strong low-end torque, good mileage, and reliable braking on slopes. "
        "I’ll use a 6-ft gyrovator/rotavator, a trolley for produce, and sometimes a reversible plough. "
        "Please suggest suitable models and the right HP range, plus tyre size/pressure and front ballast for hilly terrain. "
        "Also share service intervals, warranty, and nearby dealer guidance. Budget ≈ ₹7–9 lakh (ex-showroom)."
    )
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [ ]:
# A small bank of realistic test inputs (use one at a time with crew.kickoff), do not run
test_inputs = [
    {
        "customer": "Pathankot farmer (clay-loam, hilly patches)",
        "person": "Harpreet Singh",
        "inquiry": (
            "Fields have undulating terrain and clay-loam soil. Need high torque at low RPM and good mileage. "
            "Can I run a 6-ft gyrovator and a potato digger safely on slopes? Recommend HP range and specific models."
        ),
    },
    {
        "customer": "Kandi belt grower (prefers 4WD)",
        "person": "Gurjit Kaur",
        "inquiry": (
            "Looking for a 4WD tractor for hillside work. Which is better for slopes—synchro vs constant-mesh gearbox—and why? "
            "Please suggest HP and models that balance control and fuel efficiency."
        ),
    },
    {
        "customer": "Mixed transport + field use (≈60/40)",
        "person": "Jaspreet",
        "inquiry": (
            "Daily mandi runs (~20 km) plus rotavator work. Need mileage and comfort. "
            "Which tyres (radial vs bias) and recommended PSI for hills? Suggest suitable models."
        ),
    },
    {
        "customer": "Budget-focused buyer (≤ ₹8 lakh)",
        "person": "Manpreet",
        "inquiry": (
            "Under ₹8 lakh, prefer power steering, dual clutch, oil-immersed brakes. "
            "Will it safely pull a 1.5-ton trolley uphill? Recommend models and HP."
        ),
    },
    {
        "customer": "10-acre maize+wheat, gentle slopes",
        "person": "Balwinder",
        "inquiry": (
            "Confused between 45–50 HP vs 50–55 HP for a 5-ft rotavator on slopes. "
            "Which option is more fuel-efficient while maintaining implement performance?"
        ),
    },
    {
        "customer": "Implement compatibility check",
        "person": "Ravneet",
        "inquiry": (
            "Need to run a reversible MB plough and 7-tyne cultivator. "
            "What PTO HP is required, and do you have models with economy PTO for fuel saving on light loads?"
        ),
    },
    {
        "customer": "Service availability (Gurdaspur)",
        "person": "Amandeep",
        "inquiry": (
            "What is service and parts availability like near Gurdaspur? "
            "Estimate annual maintenance cost for ~500 hours and recommended service intervals."
        ),
    },
    {
        "customer": "Hillside stability & safety",
        "person": "Sukhdeep",
        "inquiry": (
            "For a 6-ft gyrovator on slopes, how much front ballast is recommended? "
            "Any roll-over safety features or guidelines I should follow?"
        ),
    },
    {
        "customer": "Hydraulics & control on uneven fields",
        "person": "Kamal",
        "inquiry": (
            "What lift capacity and hydraulic controls (draft/position) are required for a 6-ft gyrovator and a small baler? "
            "Suggest models suited to uneven fields."
        ),
    },
    {
        "customer": "Road speed vs fuel burn",
        "person": "Hardeep",
        "inquiry": (
            "My mix is 60% transport, 40% field. I want road speed without high fuel burn. "
            "Any models with 12F+4R or creeper options that still work well on hills?"
        ),
    },
    {
        "customer": "Cold-start & electrics",
        "person": "Daljeet",
        "inquiry": (
            "Winters get cold. Which features help with easy starts? "
            "Battery/alternator specs to support a sprayer pump and lights?"
        ),
    },
    {
        "customer": "Terrace fields maneuverability",
        "person": "Navdeep",
        "inquiry": (
            "Need compact turning radius and good ground clearance for terrace fields. "
            "Share models/specs and tyre recommendations."
        ),
    },
    {
        "customer": "Finance & warranty clarity",
        "person": "Kiran",
        "inquiry": (
            "What finance options and typical down payment are available? "
            "Detail warranty coverage for clutch and hydraulic pump, please."
        ),
    },
    {
        "customer": "Hitch category & brands",
        "person": "Amrik",
        "inquiry": (
            "Which hitch category (Cat-I vs Cat-II) should I get, and which gyrovator brands/sizes are tested compatible?"
        ),
    },
    {
        "customer": "Hilly duty maintenance plan",
        "person": "Simran",
        "inquiry": (
            "Recommend service intervals for hilly duty cycles. "
            "Any AMC packages with doorstep service I can opt into?"
        ),
    },
]


In [ ]:
# Two vernacular realism tests (same format)
inputs_hinglish = {
    "customer": "Hoshiarpur farmer (pahaadi patches)",
    "person": "Bakhtawar Singh",
    "inquiry": (
        "Bhai, thoda pahaadi area hai. 45–50 HP 4WD chahiye jo 6-ft gyrovator easily chala le aur fuel ka kharcha kam ho. "
        "Kaun-sa model lena chahiye, tyre size/pressure aur front weight kitna rakhun?"
    ),
}

inputs_punjabi = {
    "customer": "ਪਹਾੜੀ ਇਲਾਕਾ (Hoshiarpur)",
    "person": "ਬਖ਼ਤਾਵਰ ਸਿੰਘ",
    "inquiry": (
        "ਸਤ ਸ੍ਰੀ ਅਕਾਲ। 4WD ਟਰੈਕਟਰ ਚਾਹੀਦਾ ਜੋ ਮਾਈਲੇਜ ਵਧੀਆ ਦੇਵੇ ਤੇ ਢਲਾਣਾਂ ’ਚ ਸੇਫ ਹੋਵੇ। "
        "6 ਫੁੱਟ ਰੋਟਾਵੇਟਰ ਚੱਲੇਗਾ? HP, ਟਾਇਰ ਸੈੱਟਅੱਪ ਤੇ ਫਰੰਟ ਵੇਟ ਦੱਸੋ ਜੀ।"
    ),
}
